In [42]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_DIR = Path("..") / "Data"

#takes automatically the Data
message_path = next(DATA_DIR.glob("*_message_10.csv"))
orderbook_path = next(DATA_DIR.glob("*_orderbook_10.csv"))

message_path, orderbook_path

(PosixPath('../Data/GOOG_2012-06-21_34200000_57600000_message_10.csv'),
 PosixPath('../Data/GOOG_2012-06-21_34200000_57600000_orderbook_10.csv'))

In [43]:
msg_cols = ["time", "type", "order_id", "size", "price", "direction"]

msg = pd.read_csv(
    message_path,
    header=None,
    names=msg_cols,
    dtype={
        "type": "int16",
        "order_id": "int64",
        "size": "int32",
        "price": "int64",
        "direction": "int8",
    }
)

# time lo teniamo float
msg["time"] = msg["time"].astype("float64")

msg.head(20)


,time,type,order_id,size,price,direction
0,34200.015105,4,15818974,4,5794000,1
1,34200.059902,4,15818974,300,5794000,1
2,34200.113247,5,0,1,5795100,1
3,34200.113247,5,0,1,5795000,1
4,34200.113247,5,0,1,5794900,1
5,34200.113247,5,0,1,5794800,1
6,34200.113247,4,15818974,167,5794000,1
7,34200.154178,1,16155653,100,5794000,1
8,34200.173594,1,16160421,100,5791200,1
9,34200.202323,1,7091133,40,5790000,1


In [44]:
LEVELS = 10

ob_cols = []
for lvl in range(1, LEVELS + 1):
    ob_cols += [f"ask_p{lvl}", f"ask_s{lvl}", f"bid_p{lvl}", f"bid_s{lvl}"]

ob = pd.read_csv(
    orderbook_path,
    header=None,
    names=ob_cols,
    usecols=list(range(4 * LEVELS)),
)

# dtypes
for c in ob_cols:
    if c.endswith("p" + c.split("p")[-1]) and "_p" in c:  # prezzo
        ob[c] = ob[c].astype("int64")
    else:
        ob[c] = ob[c].astype("int32")

ob.head()


,ask_p1,ask_s1,bid_p1,bid_s1,ask_p2,ask_s2,bid_p2,bid_s2,ask_p3,ask_s3,...,bid_p8,bid_s8,ask_p9,ask_s9,bid_p9,bid_s9,ask_p10,ask_s10,bid_p10,bid_s10
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100


In [45]:
print(len(msg))
print(len(ob))

147916
147916


In [46]:
#b crreate a dataframe with the message file and the orderbook file
df = pd.concat([msg.reset_index(drop=True), ob.reset_index(drop=True)], axis=1)

# time è già in secondi da mezzanotte; lo mettiamo come index comodo
df = df.set_index("time", drop=True).sort_index()

df.tail()


,type,order_id,size,price,direction,ask_p1,ask_s1,bid_p1,bid_s1,ask_p2,...,bid_p8,bid_s8,ask_p9,ask_s9,bid_p9,bid_s9,ask_p10,ask_s10,bid_p10,bid_s10
time,,,,,,,,,,,,,,,,,,,,,
57599.544132,1,287157290,300,5655600,-1,5652200,42,5651200,200,5655000,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
57599.544133,1,287157291,200,5655600,-1,5652200,42,5651200,200,5655000,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
57599.544133,1,287157292,300,5655600,-1,5652200,42,5651200,200,5655000,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
57599.582154,3,287027555,100,5649700,1,5652200,42,5651200,200,5655000,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
57599.871751,3,287151822,100,5655000,-1,5652200,42,5651200,200,5655000,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [47]:
#conta i DUMMY Prima che li trasformiamo in Nan
ask_price_cols = [f"ask_p{i}" for i in range(1, LEVELS+1)]
bid_price_cols = [f"bid_p{i}" for i in range(1, LEVELS+1)]

dummy_before = (df[ask_price_cols] == ASK_DUMMY).sum().sum() + (df[bid_price_cols] == BID_DUMMY).sum().sum()
nan_before = df[ask_price_cols + bid_price_cols].isna().sum().sum()

print("Dummy prices BEFORE:", dummy_before)
print("NaN prices BEFORE:", nan_before)


Dummy prices BEFORE: 0
NaN prices BEFORE: 0


In [38]:
PRICE_SCALE = 10_000.0
ASK_DUMMY = 9_999_999_999
BID_DUMMY = -9_999_999_999

# converti top-of-book in dollari
df["ask_p1_d"] = df["ask_p1"] / PRICE_SCALE
df["bid_p1_d"] = df["bid_p1"] / PRICE_SCALE

# sostituisci dummy con NaN (così non sporcano feature multi-level)
for lvl in range(1, LEVELS + 1):
    ap, bp = f"ask_p{lvl}", f"bid_p{lvl}"
    df.loc[df[ap] == ASK_DUMMY, ap] = np.nan
    df.loc[df[bp] == BID_DUMMY, bp] = np.nan



nan_after = df[ask_price_cols + bid_price_cols].isna().sum().sum()
print("NaN prices AFTER:", nan_after)

df[["ask_p1", "ask_s1", "bid_p1", "bid_s1", "ask_p1_d", "bid_p1_d"]].head(10)


NaN prices AFTER: 0


,ask_p1,ask_s1,bid_p1,bid_s1,ask_p1_d,bid_p1_d
time,,,,,,
34200.015105,5802300.0,100,5794000.0,496,580.23,579.4
34200.059902,5802300.0,100,5794000.0,196,580.23,579.4
34200.113247,5802300.0,100,5794000.0,196,580.23,579.4
34200.113247,5802300.0,100,5794000.0,196,580.23,579.4
34200.113247,5802300.0,100,5794000.0,196,580.23,579.4
34200.113247,5802300.0,100,5794000.0,196,580.23,579.4
34200.113247,5802300.0,100,5794000.0,29,580.23,579.4
34200.154178,5802300.0,100,5794000.0,129,580.23,579.4
34200.173594,5802300.0,100,5794000.0,129,580.23,579.4


In [39]:
df["spread"] = (df["ask_p1"] - df["bid_p1"]) / PRICE_SCALE
df["mid"] = (df["ask_p1"] + df["bid_p1"]) / (2 * PRICE_SCALE)

df[["spread", "mid"]].describe()


,spread,mid
count,147916.000000,147916.000000
mean,0.296241,570.783227
std,0.148140,4.005256
min,0.010000,563.730000
25%,0.200000,567.825000
50%,0.280000,569.880000
75%,0.360000,575.255000
max,2.310000,579.815000


In [40]:
i = 100  # cambia a 10, 100, 1000...
print("MESSAGE ROW")
print(msg.iloc[i])
print("\nORDERBOOK TOP")
print(ob.iloc[i][["ask_p1", "ask_s1", "bid_p1", "bid_s1"]])


MESSAGE ROW
time         3.420603e+04
type         1.000000e+00
order_id     1.680566e+07
size         7.000000e+00
price        5.791200e+06
direction    1.000000e+00
Name: 100, dtype: float64

ORDERBOOK TOP
ask_p1    5796600
ask_s1       2300
bid_p1    5791200
bid_s1         73
Name: 100, dtype: int64
